In [34]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import e3nn_jax as e3nn
import jax
import jax.numpy as jnp
import jraph
import sys
sys.path.append('..')
import analysis
import datatypes
from input_pipeline_tf import get_datasets
from models import get_first_node_indices, segment_sample
from train import get_predictions

In [2]:
config, best_state, metrics_for_best_state, datasets = analysis.load_from_workdir('../workdirs/mace/interactions=1/l=0/channels=64')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
metrics_for_best_state

{'val': {'total_loss': array(8.657066, dtype=float32),
  'focus_loss': array(0.56171024, dtype=float32),
  'atom_type_loss': array(0.86556536, dtype=float32),
  'position_loss': array(7.229792, dtype=float32)},
 'test': {'total_loss': array(8.863261, dtype=float32),
  'focus_loss': array(0.44567674, dtype=float32),
  'atom_type_loss': array(0.8436947, dtype=float32),
  'position_loss': array(7.57389, dtype=float32)}}

In [4]:
cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

In [59]:
example_graph = next(datasets["test"].as_numpy_iterator())
frag = datatypes.Fragment.from_graphstuple(example_graph)
frag = jax.tree_map(jnp.asarray, frag)
num_nodes = frag.nodes.positions.shape[0]
num_graphs = frag.n_node.shape[0]

In [60]:
frag_unpadded = jraph.unpad_with_graphs(frag)
molecules = jraph.unbatch(frag_unpadded)

In [81]:
mol = molecules[2]

In [82]:
preds = get_predictions(best_state, mol, key)
true_focus_indices = get_first_node_indices(mol)

focus_probs = jax.nn.softmax(preds.focus_logits)
jnp.set_printoptions(precision=3, suppress=True)
focus_probs

Array([0.998, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   ], dtype=float32)

In [83]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

# covalent bond radii, in angstroms
element_radii = [0.32, 0.75, 0.71, 0.63, 0.64]

In [84]:
mol_atoms = Atoms(positions=mol.nodes.positions.tolist(), numbers=get_numbers(mol.nodes.species.tolist()))
v = view(mol_atoms, viewer='ngl')

# add focus probability highlights for each atom

for i in range(mol.n_node[0]):
    focus_prob = focus_probs.tolist()[i]
    v.view.shape.add_sphere(
        mol.nodes.positions[i].tolist(),
        [1, 0.85, 0],
        element_radii[mol.nodes.species.tolist()[i]] * 0.6,
        f"atom {i}: focus probability {focus_prob:.3f}",
    )
    v.view.update_representation(component=i+1, opacity=focus_prob)
v

# add the next atom we're adding to this molecule, predicted specie + highlight